In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from os import walk
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import metrics
import pickle

Loading the dataset and normalizing

In [ ]:
# Laod  data 
test_input = pd.read_pickle('DATA/test_input')
test_output = pd.read_pickle('DATA/test_output')
index = test_input.columns

# Normlaization of input data
# Data normalization according to training dataset/ model
filehandler = open('Weights/Norm', 'rb') 
std_scaler = pickle.load(filehandler)
inputn = pd.DataFrame(std_scaler.transform(test_input), columns=test_input.columns) 
# inputn is still a daraframe with numeric index
outputn = test_output/10**6  #  change of units, outputn is still a daraframe with time index

Define the models to be tested

In [ ]:
include_wave = 'no'
modelIDs = [0, 1, 2, 3, 4, 5]
# modelIDs = [3]
# (0 for SCADA only, 1 for SCADA+Acc17, 2 for SCADA+Acc38, 3 for SCADA+Acc77, 
# 4 for SCADA+Acc17&38, 5 for SCADA+Acc17&38&77 

Err_Mtl = np.zeros([len(modelIDs)])
Err_Mtn = np.zeros([len(modelIDs)])
for i in range(len(modelIDs)):
    modelID = modelIDs[i]
    
    # Retrive features based on the modelID
    index1 = pd.core.indexes.base.Index([]) # create a blank index array
    if include_wave == 'yes': 
        index1 = index1.append(index[0:3])
    if modelID == 1: # Acc17
        index1 = index1.append(index[[3,4,9,10,15,16]])
    if modelID == 2: # Acc38
        index1 = index1.append(index[[5,6,11,12,17,18]])
    if modelID == 3: # Acc77
        index1 = index1.append(index[[7,8,13,14,19,20]])
    if modelID == 4: # Acc17&38
        index1 = index1.append(index[[3,4,5,6,9,10,11,12,15,16,17,18]])   
    if modelID == 5: # Acc17&38&77
        index1 = index1.append(index[3:21])
    index1 = index1.append(index[21:]) # SCADA
    X = inputn[index1].values
    Y = outputn.values
    
    # Loading the model to evaluate
    VSnet = load_model('Weights/01_SensorPlacementTest/DNNModel%s_IncWave%s.h5' % (modelID, include_wave))
    
    # Prediction and Compute Error
    nn_output = VSnet.predict(X)
    Err_Mtl[i] = np.mean(abs((Y[:,0]- nn_output[:,0])/Y[:,0]))*100
    Err_Mtn[i] = np.mean(abs((Y[:,1]- nn_output[:,1])/Y[:,1]))*100   

In [ ]:
print(Err_Mtl)
print(Err_Mtn)

In [ ]:
if include_wave == 'yes':
    Models = ['SCADA+Wave', 'SCADA+Wave+Acc017', 'SCADA+Wave+Acc038', 'SCADA+Wave+Acc077', 
              'SCADA+Wave+Acc017,038', 'SCADA+Wave+Acc017,038,077']
else:
    Models = ['SCADA', 'SCADA+Acc017', 'SCADA+Acc038', 'SCADA+Acc077', 
              'SCADA+Acc017,038', 'SCADA+Acc017,038,077']

cm = 1/2.54  # centimeters in inches
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 9

x = np.arange(6)*2
fig, ax = plt.subplots(1, figsize=(8.5*cm, 8.5*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.15, right=.98, top=0.95, bottom=0.4, hspace = 0.65, wspace=0.15)
ax.bar(x, Err_Mtl, color = '#6baed6', width=-0.8, align='edge', label = "DEM$_{tl}$")
ax.bar(x, Err_Mtn,  color = '#08306b', width=0.8, align='edge', label = "DEM$_{tn}$")
ax.set_ylabel('Test error %')
ax.set_xticks(x) 
ax.set_xticklabels(Models, horizontalalignment= 'right', rotation=45)
ax.set_ylim([0, 28])
def add_value_label(x_list,y_list):
    for i in range(1, len(x_list)+1):
        plt.text(x_list[i-1],y_list[i-1]+0.5,y_list[i-1], ha="center", fontsize = 8)

add_value_label(x-0.4,np.around(Err_Mtl,1))
add_value_label(x+0.4,np.around(Err_Mtn,1))

ax.legend(loc ="upper right", fontsize=9)
# plt.grid(axis='y', color='0.95')
fig.savefig('Figures/01_SensorPlacementTest/DNNErrors_IncWave%s.pdf' % (include_wave))